In [1]:
import tensorflow as tf
import numpy as np


In [2]:
inp = tf.random_uniform([1 , 299 , 299 , 3])

In [7]:
def conv1(name , data , f , k1 , k2 , s1=1 , s2=1 , padding_val = 'SAME'):
    
    with tf.variable_scope(name , reuse = tf.AUTO_REUSE) as scope:
            
        conv = tf.layers.conv2d(
        inputs=data,
        kernel_size=[k1,k2],
        filters= f,
        strides=(s1,s2),
        activation=tf.nn.relu,
        padding=padding_val)
 
    return conv

In [8]:
def conv(data , s , f1 , f2 , padding_val , name ):
    
    with tf.variable_scope(name , reuse=tf.AUTO_REUSE) as scope:
        
        out = tf.layers.conv2d(
        inputs=data,
        filters=f1,
        kernel_size = [s,s] ,
        activation=tf.nn.relu)

        out = tf.layers.conv2d(
        inputs=out,
        filters=f2,
        padding = padding_val,
        kernel_size=[3,3],
        activation=tf.nn.relu)

        out = tf.layers.max_pooling2d(
        inputs=out,
        pool_size=[3,3],
        strides=(2,2))
        
    return out

In [9]:
def inception1(data , name ):
    
    with tf.variable_scope(name , reuse=tf.AUTO_REUSE) as scope:
        
        out1 = conv1(name+'out1' , data , 64 , 1 , 1 )
        
        out2 = conv1(name+'out2'  , data , 48 , 1 , 1)
        out2 = conv1(name+'out21' , out2 , 64 , 5 , 5)
        
        out3= conv1(name+'out3'   , data , 64 , 1 , 1)
        out3= conv1(name+'out31'  , data , 96 , 3 , 3)
        out3= conv1(name+'out32'  , data , 96 , 3 , 3)
        
        out4= tf.layers.average_pooling2d(
        inputs=data,
        pool_size=[3,3],
        strides=(1,1),
        padding='SAME',
        name=name+'pool1')
        
        if name == 'inception1_0':
            out4= conv1(name+'out4' , out4 , 32 , 1 , 1)
        else:
            out4= conv1(name+'out4' , out4 , 64 , 1 , 1)
        
    out = tf.concat([out1,out2,out3,out4] , axis=3)
        
    return out

In [10]:
def transform1(data , name):
    
    with tf.variable_scope(name , reuse=tf.AUTO_REUSE) as scope:
        
        temp1= conv1(name+'temp1' , data , 384 , 3 , 3 , 2 , 2 , padding_val = 'VALID' )
        
        temp2= conv1(name+'temp2' , data , 64 , 1 , 1 )
        temp2= conv1(name+'temp21', temp2, 96 , 3 , 3 )
        temp2= conv1(name+'temp22', temp2, 96 , 3 , 3 , 2 , 2 , padding_val = 'VALID')
        
        temp3 = tf.layers.max_pooling2d(
        inputs=data,
        pool_size=[3,3],
        strides=(2,2),
        name=name+'temp3')
        
        data = tf.concat([temp1 , temp2 , temp3 ] , axis = 3)
        
    return data

In [11]:
def transform2(data , name):
    
    with tf.variable_scope(name , reuse=tf.AUTO_REUSE) as scope:
        
        temp= conv1(name+'temp1'  , data , 192 , 1 , 1 )
        temp1= conv1(name+'temp11', temp , 320 , 3 , 3 , 2 , 2 , padding_val = 'VALID')

        temp2= conv1(name+'temp21', temp, 192 , 1 , 7 )
        temp2= conv1(name+'temp22', temp2, 192 , 7 , 1 )
        temp2= conv1(name+'temp23', temp2, 192 , 3 , 3 , 2 , 2 , padding_val = 'VALID')
        
        temp3 = tf.layers.max_pooling2d(
        inputs=data,
        pool_size=[3,3],
        strides=(2,2),
        name=name+'temp3')
        
        data = tf.concat([temp1 , temp2 , temp3 ] , axis = 3)
        
    return data

In [12]:
def inception2(data , name):

    f=160
    if name == 'inception2_0':
        f = 128
    elif name == 'inception2_3':
        f = 192
        
    with tf.variable_scope(name , reuse=tf.AUTO_REUSE) as scope:
        
        out1 = conv1(name+'out1' , data , 192 , 1 , 1 )
        
        out = conv1(name+'out2'   , data ,  f  , 1 , 1 )
        out2 = conv1(name+'out21' , out  ,  f  , 1 , 7 )
        out2 = conv1(name+'out22' , out2 , 192 , 7 , 1 )
        
        out3 = conv1(name+'out3'  , out  , f   , 7 , 1 )
        out3 = conv1(name+'out31' , out3 , f   , 1 , 7 )
        out3 = conv1(name+'out32' , out3 , f   , 7 , 1 )
        out3 = conv1(name+'out33' , out3 , 192 , 1 , 7 )
        
        out4= tf.layers.average_pooling2d(
        inputs=data,
        pool_size=[3,3],
        strides=(1,1),
        padding='SAME',
        name=name+'pool1')
        
        out4= conv1(name+'out4' , out4 , 192 , 1 , 1 )
        
        out = tf.concat([out1 , out2 , out3 , out4] , axis = 3)
        
    return out
            

In [13]:
def inception3(data , name ):
    
    with tf.variable_scope(name , reuse=tf.AUTO_REUSE) as scope:
        
        out1= conv1(name+'out1' , data , 320 , 1 , 1 )
        
        temp1= conv1(name+'temp1' ,  data , 384 , 1 , 1 )
        out2= conv1(name+'out2'   , temp1 , 384 , 1 , 3 )
        out3= conv1(name+'out21'  , temp1 , 384 , 3 , 1 )
        
        temp2= conv1(name+'temp2'  , data  , 448 , 1 , 1 )
        temp2= conv1(name+'temp21' , temp2 , 384 , 3 , 3 )
        out4 = conv1(name+'out4'   , temp2 , 384 , 1 , 3 )
        out5 = conv1(name+'out5'   , temp2 , 384 , 3 , 1 )
         
        out6= tf.layers.average_pooling2d(
        inputs=data,
        pool_size=[3,3],
        strides=(1,1),
        padding='SAME',
        name=name+'pool1')
        
        out6= conv1(name+'out6' , data , 192 , 1 , 1 )
    
    out = tf.concat([out1 , out2 , out3 , out4 , out5 , out6] , axis = 3)
    
    return out

In [14]:
def model (data):
    
    out = tf.layers.conv2d(
    inputs=data,
    filters=32,
    kernel_size=[3,3],
    strides=(2,2),
    activation=tf.nn.relu,
    reuse=tf.AUTO_REUSE,
    name='DATA')
    
    print(out)
    
    out = conv(out , 3 , 32 , 64 ,padding_val= 'SAME' , name= 'CONV1')
    
    print(out)
    
    out = conv(out , 1 , 80 , 192 ,padding_val= 'VALID' ,name= 'CONV2')
    
    print(out)
    
    for i in range (3):
        with tf.variable_scope('inception1') as scope:
            
            out = inception1(out , 'inception1_{}'.format(i))
            
    print(out)
    
    out = transform1(out , 'transform1')

    for i in range (4):
        with tf.variable_scope('inception2') as scope:
            
            out = inception2(out , 'inception2_{}'.format(i))
     
    print(out)
    
    out = transform2(out , 'transform2')
    
    for i in range (2):
        with tf.variable_scope('inception3') as scope:
            
            out = inception3(out , 'inception3_{}'.format(i))
        
    print(out)
    
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    
    out = tf.layers.dense(
    inputs= out ,
    units = 1000,
    reuse = tf.AUTO_REUSE ,
    name = 'out_layer')
    
    return out

In [11]:
output = model(inp)
output

Tensor("DATA/Relu:0", shape=(1, 149, 149, 32), dtype=float32)
Tensor("CONV1/max_pooling2d/MaxPool:0", shape=(1, 73, 73, 64), dtype=float32)
Tensor("CONV2/max_pooling2d/MaxPool:0", shape=(1, 35, 35, 192), dtype=float32)
Tensor("inception1_2/concat:0", shape=(1, 35, 35, 288), dtype=float32)
Tensor("inception2_3/inception2_3/concat:0", shape=(1, 17, 17, 768), dtype=float32)
Tensor("inception3_1/concat:0", shape=(1, 8, 8, 2048), dtype=float32)


<tf.Tensor 'out_layer/BiasAdd:0' shape=(1, 1000) dtype=float32>